<a href="https://colab.research.google.com/github/CSCCNY/final-project-recomposeclassics/blob/main/BaselineOnMidiFiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Recompose Classics : Baseline Model on Midi files**
######Hannah Do

---
***Summary***

1. Extract the notes and chords from the midi files with **music21**
2. Build Baseline Model
3. Run the models to predict and evaluate


---

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [274]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord

import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D


#### 1. Extract the notes and chords from the midi files with **music21**

In [218]:
def get_notes():

    notes = []

    for file in glob.glob("/content/gdrive/My Drive/RecomposeClassics/chopin/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/gdrive/My Drive/RecomposeClassics/data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes


def prepare_sequences(notes, n_vocab):

    sequence_length = 50

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, sequence_length):

        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])

    n_patterns = len(network_input)


    
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    # network_output = np_utils.to_categorical(network_output)

    return network_input


In [5]:
notes = get_notes()

Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chp_op18.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p10.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p11.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chp_op31.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p1.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p12.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p14.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p15.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p13.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p18.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p16.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p17.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p2.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p19.mid
Parsing 

In [219]:
from keras.utils import np_utils

n_vocab = len(set(notes))

network_input = prepare_sequences(notes, n_vocab)

In [220]:
np.shape(network_input)

(1268, 50, 1)

In [241]:
a, b = np.split(network_input, 2)

np.shape(a)


(634, 50, 1)

In [242]:
c, d = np.split(a, 2)

np.shape(c)

(317, 50, 1)

In [243]:
input1 = np.concatenate(((b, d)), axis=0)

input2 = c

In [244]:

input1 = input1.transpose(1, 2, 0)
np.shape(input1)

(50, 1, 951)

In [245]:
input2 = input2.transpose(1, 2, 0)

In [246]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(input1, input2, test_size=0.2, random_state=42)

In [247]:
np.shape(x_train)

(40, 1, 951)

In [278]:

def baseline(network_input, n_vocab):

    model = Sequential()

    model.add(Dense(2, input_shape=(np.shape(network_input))))
    model.add(Dense(3))
    model.add(Dense(4))
    # model.add(Dropout(0.2))
    model.add(Dense(317))

    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model
    

In [280]:
baseline_model = baseline(x_train, n_vocab)


In [281]:
baseline_model.summary()

Model: "sequential_51"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_139 (Dense)            (None, 40, 1, 2)          1904      
_________________________________________________________________
dense_140 (Dense)            (None, 40, 1, 3)          9         
_________________________________________________________________
dense_141 (Dense)            (None, 40, 1, 4)          16        
_________________________________________________________________
dense_142 (Dense)            (None, 40, 1, 317)        1585      
_________________________________________________________________
activation_53 (Activation)   (None, 40, 1, 317)        0         
Total params: 3,514
Trainable params: 3,514
Non-trainable params: 0
_________________________________________________________________


In [288]:
from keras.callbacks import ModelCheckpoint

def train(baseline_model, input1, input2):

    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    baseline_model.fit(input1, input2, epochs=250, batch_size=128, callbacks=callbacks_list)

    baseline_model.compile(
      optimizer=tf.optimizers.Adam(learning_rate=0.1),
      loss='mean_absolute_error', metrics='mean_squared_error')

In [289]:

train(baseline_model, x_train, y_train)


Epoch 1/250
1/1 [==============================] - 0s 13ms/step - loss: 0.7261 - mean_squared_error: 0.6033
Epoch 2/250
1/1 [==============================] - 0s 7ms/step - loss: 0.7261 - mean_squared_error: 0.6033
Epoch 3/250
1/1 [==============================] - 0s 6ms/step - loss: 0.7261 - mean_squared_error: 0.6033
Epoch 4/250
1/1 [==============================] - 0s 8ms/step - loss: 0.7261 - mean_squared_error: 0.6033
Epoch 5/250
1/1 [==============================] - 0s 7ms/step - loss: 0.7261 - mean_squared_error: 0.6033
Epoch 6/250
1/1 [==============================] - 0s 7ms/step - loss: 0.7261 - mean_squared_error: 0.6033
Epoch 7/250
1/1 [==============================] - 0s 8ms/step - loss: 0.7261 - mean_squared_error: 0.6033
Epoch 8/250
1/1 [==============================] - 0s 8ms/step - loss: 0.7261 - mean_squared_error: 0.6033
Epoch 9/250
1/1 [==============================] - 0s 6ms/step - loss: 0.7261 - mean_squared_error: 0.6033
Epoch 10/250
1/1 [==================

In [ ]:
# all-in-one function

def train_network():

    notes = get_notes()

    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = baseline(network_input, n_vocab)

    train(model, network_input, network_output)

###References
- Classical Piano Composer by Skuldur : https://github.com/Skuldur/Classical-Piano-Composer


